!curl -O https://code.s3.yandex.net/data-scientist/ds-plus-final.db

In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# Путь к файлу базы данных
path_to_db = 'ds-plus-final.db'

# Создание подключения к базе данных
engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

# Получение списка таблиц в базе данных
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", engine)
display(tables)

,name
0,data_arc
1,data_bulk
2,data_bulk_time
3,data_wire_time
4,contract
5,personal
6,phone
7,internet
8,data_gas
9,data_temp


In [6]:
# Список нужных таблиц
required_tables = ['contract', 'personal', 'internet', 'phone']

# Загрузка данных из нужных таблиц
data = {}
for table in required_tables:
    data[table] = pd.read_sql(f"SELECT * FROM {table}", engine)

# Просмотр первых строк каждой таблицы
for table_name, df in data.items():
    print(f"Таблица: {table_name}")
    display(df.head())

Таблица: contract


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.3,1960.6
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.7,353.5


Таблица: personal


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


Таблица: internet


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


Таблица: phone


,CustomerId,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


In [7]:
# Проверка информации о каждой таблице
for table_name, df in data.items():
    print(f"Информация о таблице: {table_name}")
    print(df.info())
    print("\n")

Информация о таблице: contract
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        7043 non-null   object
 1   BeginDate         7043 non-null   object
 2   EndDate           7043 non-null   object
 3   Type              7043 non-null   object
 4   PaperlessBilling  7043 non-null   object
 5   PaymentMethod     7043 non-null   object
 6   MonthlyCharges    7043 non-null   object
 7   TotalCharges      7043 non-null   object
dtypes: object(8)
memory usage: 440.3+ KB
None


Информация о таблице: personal
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   object
 3


---

### Анализ загруженных данных

#### 1. **Таблица `contract`**
- **Количество строк**: 7043
- **Особенности**:
  - Все столбцы имеют тип `object`, хотя некоторые из них (например, `MonthlyCharges` и `TotalCharges`) должны быть числовыми.
  - В столбце `EndDate` значение "No" указывает, что клиент все еще активен.
  - Столбец `BeginDate` содержит даты начала договоров.

#### 2. **Таблица `personal`**
- **Количество строк**: 7043
- **Особенности**:
  - Все столбцы также имеют тип `object`.
  - Столбец `SeniorCitizen` (является ли клиент пенсионером) имеет значения "0" или "1", которые можно преобразовать в числовой тип.
  - Столбцы `Partner` и `Dependents` содержат значения "Yes"/"No".

#### 3. **Таблица `internet`**
- **Количество строк**: 5517
- **Особенности**:
  - Таблица содержит информацию только о клиентах, которые пользуются интернет-услугами.
  - Все столбцы имеют тип `object`.
  - Столбцы `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies` содержат значения "Yes"/"No".

#### 4. **Таблица `phone`**
- **Количество строк**: 6361
- **Особенности**:
  - Таблица содержит информацию только о клиентах, которые пользуются телефонными услугами.
  - Столбец `MultipleLines` содержит значения "Yes"/"No".

---

### Наблюдения
1. **Размеры таблиц различаются**:
   - Таблица `contract` содержит данные обо всех клиентах (7043 строки).
   - Таблицы `internet` и `phone` содержат меньше строк, так как не все клиенты пользуются интернетом или телефонией.
   - Это может привести к проблемам при объединении таблиц. Например, для клиентов без интернета или телефона в соответствующих таблицах не будет записей.

2. **Пропущенные значения**:
   - Прямых пропусков (`NaN`) нет, но отсутствие данных в таблицах `internet` и `phone` можно интерпретировать как отсутствие соответствующих услуг у клиента.

3. **Типы данных**:
   - Многие столбцы имеют тип `object`, хотя логичнее было бы использовать числовые типы (например, для `MonthlyCharges`, `TotalCharges`, `SeniorCitizen`).

---

### Что дальше?
На основе этого анализа мы можем перейти к **шагу 2** — исследовательскому анализу и предобработке данных. Вот план действий:

1. **Обработка типов данных**:
   - Преобразовать `MonthlyCharges` и `TotalCharges` в числовой формат.
   - Преобразовать бинарные столбцы (`SeniorCitizen`, `Partner`, `Dependents`, `PaperlessBilling`, и другие с "Yes"/"No") в числовые значения.

2. **Обработка дат**:
   - Преобразовать `BeginDate` и `EndDate` в формат даты.
   - Для активных клиентов (`EndDate = "No"`) заменить значение на текущую дату.

3. **Объединение таблиц**:
   - Объединить все таблицы по `customerID`.
   - Добавить признаки для клиентов, которые не пользуются интернетом или телефонией.

4. **Генерация новых признаков**:
   - Рассчитать длительность контракта.
   - Создать целевую переменную на основе `EndDate`.

5. **Исследовательский анализ данных**:
   - Проверить распределение признаков.
   - Визуализировать соотношение ушедших и оставшихся клиентов.
   - Провести корреляционный анализ.

In [8]:
RANDOM_STATE = 170225